# TARGET DETERMINATION FOR PIP MINER MODEL

This experiment is an extension of the `parameters` experiment. Given the range of with stable Martin Ratio:
- what cluster identity should be selected? How can we combine them into a strategy?
- what could be the exit strategy for the strategy?

In [1]:
# Import Necessary Libraries, Define the parameters
import logging
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import quantstats as qt
import seaborn as sns
from quantminer import Miner

logger = logging.getLogger('optuna')
logger.setLevel(logging.WARNING)

data_dir = Path.cwd().parent / 'data'

### STEP 0 : DATA PREPARATION AND MODEL TRAINING
- Asset : EURUSD, 1-hour
- Parameter
  - n_pivots : 3; 4
  - n_clusters : 16; 15 
  - n_lookback : 8; 14
  - hold_period : 2, 3

In [2]:
# Read Price Data
data_path = data_dir / 'eur_h1.parquet'
raw_data = pd.read_parquet(data_path)

# Clean the data
data = raw_data.copy()
data = data.dropna()

# Feature Engineering
data['returns'] = data['close'].diff().fillna(0)
data['returns+1'] = data['returns'].shift(-1)

# Prepare the training data
train_daterange = pd.date_range('2010-01-01', '2021-12-31', freq='1h')
train_df = data[data.index.isin(train_daterange)]
train_data = np.array(train_df['close'])

In [3]:
# Parameters
n_pivots=3
n_clusters = 24
n_lookback=15
hold_period=3

miner = Miner(
    n_pivots=n_pivots,
    n_clusters=n_clusters,
    n_lookback=n_lookback,
    hold_period=hold_period,
    model_type='sequential'
)

# Fit the model
miner.fit(train_data)

11.722329145998637

In [4]:
# Create a feature for the predicted labels
data['cluster_labels'] = miner.transform(data['close']).astype(int)
train_df = data[data.index.isin(train_daterange)]

In [5]:
# Fixed Parameters
# fig_base = go.Figure()
# for _ in range(n_clusters):
#     fig_base.add_trace(go.Scatter(x=train_df.index, y=train_df['close'], mode='lines'))
# fig_base.show()